<a href="https://colab.research.google.com/github/PatrickRosa1/Master-s-degree/blob/main/PKL_to_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
import pickle

# Substitua pelo caminho do seu arquivo .pkl no Google Drive
with open('/content/drive/MyDrive/Mestrado/Arvores/HM.pkl', 'rb') as file:
    clf = pickle.load(file)

In [ ]:
!pip install joblib

import joblib

# Save the fitted model using joblib
# Replace 'your_fitted_model' with the actual model object
# joblib.dump(your_fitted_model, '/content/drive/MyDrive/Mestrado/Arvores/HM.pkl')

# Load the model using joblib
with open('/content/drive/MyDrive/Mestrado/Arvores/HM.pkl', 'rb') as file:
    clf = joblib.load(file)

In [ ]:
import numpy as np

# Verificar o tipo do objeto carregado
print(type(clf))

# Se for um numpy.ndarray, verifique seu conteúdo
if isinstance(clf, np.ndarray):
    print("O objeto é um array numpy. Tamanho:", clf.shape)
    print("Primeiro elemento do array:", clf[0])

<class 'numpy.ndarray'>
O objeto é um array numpy. Tamanho: (24,)
Primeiro elemento do array: rd


In [ ]:
# Verificar se o objeto é um ensemble (por exemplo, RandomForestClassifier)
if hasattr(clf, 'estimators_'):
    print("O objeto é um ensemble de árvores.")
    # Acessar a primeira árvore do ensemble
    tree = clf.estimators_[0]
    # Gerar o código C para a primeira árvore
    tree_to_c(tree, feature_names, 'arvore_decisao.c')
else:
    print("O objeto não é um ensemble. Verifique o tipo do objeto.")

O objeto não é um ensemble. Verifique o tipo do objeto.


In [ ]:
import joblib
import numpy as np
from sklearn.tree import _tree

# Função para converter a árvore de decisão em código C
def tree_to_c(tree, feature_names, file_name):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    with open(file_name, 'w') as f:
        f.write("#include <stdio.h>\n\n")
        f.write("int predict(double *features) {\n")

        def recurse(node, depth):
            indent = "    " * depth
            if tree_.feature[node] != _tree.TREE_UNDEFINED:
                name = feature_name[node]
                threshold = tree_.threshold[node]
                f.write(f"{indent}if (features[{tree_.feature[node]}] <= {threshold}) {{\n")
                recurse(tree_.children_left[node], depth + 1)
                f.write(f"{indent}}} else {{\n")
                recurse(tree_.children_right[node], depth + 1)
                f.write(f"{indent}}}\n")
            else:
                f.write(f"{indent}return {int(tree_.value[node].argmax())};\n")

        recurse(0, 1)
        f.write("}\n\n")
        f.write("int main() {\n")
        f.write("    double features[{}];\n".format(len(feature_names)))
        f.write("    // Preencha o array features com os valores das características\n")
        f.write("    int prediction = predict(features);\n")
        f.write("    printf(\"Predição: %d\\n\", prediction);\n")
        f.write("    return 0;\n")
        f.write("}\n")

# Carregar o modelo salvo
with open('/content/drive/MyDrive/Mestrado/Arvores/RR1.pkl', 'rb') as file:
    clf = joblib.load(file)

# Verificar o tipo do objeto carregado
print("Tipo do objeto carregado:", type(clf))

# Nomes das características (features)
feature_names = [  "rd", "switchable_rate", "switchable_ctx_0", "switchable_ctx_1", "need_search",
    "bsize", "partition", "mode", "uv_mode", "current_qindex", "default_interp_skip_flags",
    "interp_filter_search_mask", "mi_row", "mi_col", "width", "height", "mb_to_bottom_edge",
    "mb_to_left_edge", "mb_to_right_edge", "mb_to_top_edge", "rd_stats_luma_rate",
    "rd_stats_luma_dist", "rd_stats_luma_sse", "rd_stats_luma_skip_txfm", "rd_stats_rate",
    "rd_stats_dist", "rd_stats_sse", "rd_stats_skip_txfm"]  # Substitua pelos nomes reais

# Since clf is a DecisionTreeClassifier, call tree_to_c directly
tree_to_c(clf, feature_names, 'all.c') # Call tree_to_c with the loaded model

# Verificar o conteúdo do arquivo gerado
#with open('arvore_decisao.c', 'r') as f:
    #print(f.read())

Tipo do objeto carregado: <class 'sklearn.tree._classes.DecisionTreeClassifier'>


In [ ]:
from google.colab import files
files.download('all.c')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>